# Iteration 16: random number streams (3) stage 2

## Plain English Summary



## Notebook imports

In [4]:
from IPython.display import Markdown, display

import numpy as np
import pandas as pd
import math

import simpy

## LLM Prompt

In [ ]:
display(Markdown("../01_perplexity/inputs/iteration_16.md"))

## Date and time of input to LLM

> 1830 14th December 2023 

## Sources

* https://www.nature.com/articles/s41598-022-25689-y
* https://towardsdatascience.com/object-oriented-discrete-event-simulation-with-simpy-53ad82f5f6e2
* https://stackoverflow.com/questions/29012212/implementing-common-random-numbers-in-a-simulation
* https://www.mathworks.com/help/matlab/math/creating-and-controlling-a-random-number-stream.html

## Observations on the LLM Output

* The random number streams (rng_*) are now defined and initialized in the Experiment class.
* The setup_streams method in Experiment creates the random number streams using a master stream and seeds.
* NOTE: In the CCU class, the patient generator functions (*_arrivals) are using the corresponding random number stream from the Experiment instance. the rng for elective_treatment and unplanned_treatment have been removed.
* The rng_* attributes have been removed from the CCU class.


## LLM Output 1:

In [ ]:
class Experiment:
    def __init__(self, accident_emergency_arrival_rate=22.72, wards_arrival_rate=26.0, emergency_surgery_arrival_rate=37.0, other_hospitals_arrival_rate=47.2, xray_department_arrival_rate=575.0, elective_surgery_arrival_rate=17.91, elective_surgery_arrival_std_dev=3.16, accident_emergency_lognormal_mu=128.79, accident_emergency_lognormal_sigma=267.51, wards_lognormal_mu=177.89, wards_lognormal_sigma=276.54, emergency_surgery_lognormal_mu=140.15, emergency_surgery_lognormal_sigma=218.02, other_hospitals_lognormal_mu=212.86, other_hospitals_lognormal_sigma=457.67, xray_department_lognormal_mu=87.53, xray_department_lognormal_sigma=108.67, elective_surgery_treatment_rate=57.34, num_critical_care_beds=24, intensive_cleaning_duration=5, results_collection_period=12 * 30 * 24, warm_up_period=30 * 24, trace=False, random_number_set=0):
        # ... (other attributes)
        self.rng_accident_emergency = np.random.default_rng()
        self.rng_wards = np.random.default_rng()
        self.rng_emergency_surgery = np.random.default_rng()
        self.rng_other_hospitals = np.random.default_rng()
        self.rng_xray_department = np.random.default_rng()
        self.rng_elective_surgery = np.random.default_rng()
        self.setup_streams(random_number_set)

    def setup_streams(self, random_number_set):
        master_stream = np.random.default_rng(random_number_set)
        seeds = master_stream.integers(0, np.iinfo(np.int64).max, 12, dtype=np.int64)
        self.rng_accident_emergency = np.random.default_rng(seeds[0])
        self.rng_wards = np.random.default_rng(seeds[1])
        self.rng_emergency_surgery = np.random.default_rng(seeds[2])
        self.rng_other_hospitals = np.random.default_rng(seeds[3])
        self.rng_xray_department = np.random.default_rng(seeds[4])
        self.rng_elective_surgery = np.random.default_rng(seeds[5])
        # ... (other methods)

class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        # ... (same method)

    def warmup_complete(self):
        # ... (same method)

    def reset_kpi(self):
        # ... (same method)

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        # ... (same method using experiment.rng_* streams)

    def elective_admissions_process(self):
        # ... (same method)

    def run(self):
        # ... (same method)



## Additional prompt: 

Modify the modified code as follows. In the CCU class, all the elective_admissions_process and all the unplanned_admissions_process should use the random number generators  self.rng_elective_treatment and self.rng_unplanned_treatment respectively

The key changes are:
In the unplanned_admissions_process method, the self.experiment.rng_unplanned_treatment stream is used for sampling the length of stay distributions for all unplanned admission sources.
In the elective_admissions_process method, the self.experiment.rng_elective_treatment stream is used for sampling the length of stay distribution for elective admissions.
All other parts of the code remain the same.

> The code returned an indentation error in the treatment functions which had to be corrected (picked up by tester)

> Patient_id is incrementing after the print statements.  It doesn't affect the results but the print messages don't make sense - this isn't corrected here.

In [12]:
import numpy as np
import pandas as pd

def multiple_replications(experiment, num_replications=5):
    all_results = []
    for i in range(num_replications):
        experiment.setup_streams(i)  # Call setup_streams method
        env = simpy.Environment()
        ccu = CCU(env, experiment)
        ccu.reset_kpi()
        results = ccu.run()
        results['Replication'] = i + 1
        all_results.append(results)
    combined_results = pd.concat(all_results, ignore_index=True)
    return combined_results

def results_summary(results_df):
    # Drop the replication column
    results_df = results_df.drop('Replication', axis=1)

    # Calculate the mean and standard deviation of each column
    summary_df = results_df.describe().loc[['mean', 'std']].T

    return summary_df


class Experiment:
    def __init__(self, 
                 accident_emergency_arrival_rate=22.72, 
                 wards_arrival_rate=26.0, 
                 emergency_surgery_arrival_rate=37.0, 
                 other_hospitals_arrival_rate=47.2, 
                 xray_department_arrival_rate=575.0, 
                 elective_surgery_arrival_rate=17.91, 
                 elective_surgery_arrival_std_dev=3.16, 
                 accident_emergency_lognormal_mu=128.79, 
                 accident_emergency_lognormal_sigma=267.51, 
                 wards_lognormal_mu=177.89, 
                 wards_lognormal_sigma=276.54, 
                 emergency_surgery_lognormal_mu=140.15, 
                 emergency_surgery_lognormal_sigma=218.02, 
                 other_hospitals_lognormal_mu=212.86, 
                 other_hospitals_lognormal_sigma=457.67, 
                 xray_department_lognormal_mu=87.53, 
                 xray_department_lognormal_sigma=108.67, 
                 elective_surgery_treatment_rate=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 results_collection_period=12 * 30 * 24, 
                 warm_up_period=30 * 24, 
                 trace=False, 
                 random_number_set=0):
        self.accident_emergency_arrival_rate = accident_emergency_arrival_rate
        self.wards_arrival_rate = wards_arrival_rate
        self.emergency_surgery_arrival_rate = emergency_surgery_arrival_rate
        self.other_hospitals_arrival_rate = other_hospitals_arrival_rate
        self.xray_department_arrival_rate = xray_department_arrival_rate
        self.elective_surgery_arrival_rate = elective_surgery_arrival_rate
        self.elective_surgery_arrival_std_dev = elective_surgery_arrival_std_dev
        self.accident_emergency_lognormal_mu = accident_emergency_lognormal_mu
        self.accident_emergency_lognormal_sigma = accident_emergency_lognormal_sigma
        self.wards_lognormal_mu = wards_lognormal_mu
        self.wards_lognormal_sigma = wards_lognormal_sigma
        self.emergency_surgery_lognormal_mu = emergency_surgery_lognormal_mu
        self.emergency_surgery_lognormal_sigma = emergency_surgery_lognormal_sigma
        self.other_hospitals_lognormal_mu = other_hospitals_lognormal_mu
        self.other_hospitals_lognormal_sigma = other_hospitals_lognormal_sigma
        self.xray_department_lognormal_mu = xray_department_lognormal_mu
        self.xray_department_lognormal_sigma = xray_department_lognormal_sigma
        self.elective_surgery_treatment_rate = elective_surgery_treatment_rate
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.results_collection_period = results_collection_period
        self.warm_up_period = warm_up_period
        self.total_run_length = self.warm_up_period + self.results_collection_period
        self.trace = trace
        self.random_number_set = random_number_set
        self.rng_accident_emergency = np.random.default_rng()
        self.rng_wards = np.random.default_rng()
        self.rng_emergency_surgery = np.random.default_rng()
        self.rng_other_hospitals = np.random.default_rng()
        self.rng_xray_department = np.random.default_rng()
        self.rng_elective_surgery = np.random.default_rng()
        self.rng_elective_treatment = np.random.default_rng()
        self.rng_unplanned_treatment = np.random.default_rng()
        self.setup_streams(random_number_set)

    def setup_streams(self, random_number_set):
        master_stream = np.random.default_rng(random_number_set)
        seeds = master_stream.integers(0, np.iinfo(np.int64).max, 12, dtype=np.int64)
        self.rng_accident_emergency = np.random.default_rng(seeds[0])
        self.rng_wards = np.random.default_rng(seeds[1])
        self.rng_emergency_surgery = np.random.default_rng(seeds[2])
        self.rng_other_hospitals = np.random.default_rng(seeds[3])
        self.rng_xray_department = np.random.default_rng(seeds[4])
        self.rng_elective_surgery = np.random.default_rng(seeds[5])
        self.rng_unplanned_treatment = np.random.default_rng(seeds[6])
        self.rng_elective_treatment = np.random.default_rng(seeds[7])
        

In [13]:
class CCU:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_id_counter = 0
        self.cancelled_operations = 0
        self.warmup_end_time = self.experiment.warm_up_period
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.critical_care_beds = simpy.Resource(env, capacity=self.experiment.num_critical_care_beds)

    def lognormal_to_normal(self, mu, sigma):
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def warmup_complete(self):
        yield self.env.timeout(self.warmup_end_time)
        self.patient_id_counter = 0
        if self.experiment.trace:
            print("Warm-up complete")

    def reset_kpi(self):
        """Reset all performance measures to their original values"""
        self.cancelled_operations = 0
        self.total_unplanned_waiting_time = 0
        self.total_unplanned_admissions = 0
        self.total_treatment_time = 0
        self.patient_id_counter = 0  # Reset patient_id_counter

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_accident_emergency.exponential(self.experiment.accident_emergency_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Accident and Emergency at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_wards.exponential(self.experiment.wards_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the Wards at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_emergency_surgery.exponential(self.experiment.emergency_surgery_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Emergency Surgery at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_other_hospitals.exponential(self.experiment.other_hospitals_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from Other Hospitals at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_xray_department.exponential(self.experiment.xray_department_arrival_rate))
            if self.experiment.trace:
                print(f"Patient {self.patient_id_counter} arrived from the X-Ray Department at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.unplanned_admissions_process("X-Ray Department"))

    def elective_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.experiment.rng_elective_surgery.normal(self.experiment.elective_surgery_arrival_rate, self.experiment.elective_surgery_arrival_std_dev))
            if self.experiment.trace:
                print(f"Elective Patient {self.patient_id_counter} arrived at {self.env.now:.2f} hours")
            self.patient_id_counter += 1
            self.env.process(self.elective_admissions_process())

    def unplanned_admissions_process(self, source):
        patient_id = self.patient_id_counter
        if self.experiment.trace:
            print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        waiting_time = self.env.now
        with self.critical_care_beds.request() as req:
            yield req
            waiting_time = self.env.now - waiting_time
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} waited {waiting_time:.2f} hours")
                print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.experiment.accident_emergency_lognormal_mu, self.experiment.accident_emergency_lognormal_sigma)
                length_of_stay = self.experiment.rng_unplanned_treatment.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.experiment.wards_lognormal_mu, self.experiment.wards_lognormal_sigma)
                length_of_stay = self.experiment.rng_unplanned_treatment.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.experiment.emergency_surgery_lognormal_mu, self.experiment.emergency_surgery_lognormal_sigma)
                length_of_stay = self.experiment.rng_unplanned_treatment.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.experiment.other_hospitals_lognormal_mu, self.experiment.other_hospitals_lognormal_sigma)
                length_of_stay = self.experiment.rng_unplanned_treatment.lognormal(mu, sigma)
            else:
                mu, sigma = self.lognormal_to_normal(self.experiment.xray_department_lognormal_mu, self.experiment.xray_department_lognormal_sigma)
                length_of_stay = self.experiment.rng_unplanned_treatment.lognormal(mu, sigma)
            yield self.env.timeout(length_of_stay)
            if self.env.now >= self.warmup_end_time:
                self.total_treatment_time += length_of_stay
                self.total_unplanned_waiting_time += waiting_time
                self.total_unplanned_admissions += 1
            if self.experiment.trace:
                print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            if self.experiment.trace:
                print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def elective_admissions_process(self):
        patient_id = self.patient_id_counter
        if self.critical_care_beds.count == self.critical_care_beds.capacity:
            if self.env.now >= self.warmup_end_time:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds")
                self.cancelled_operations += 1
            else:
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} operation cancelled at {self.env.now:.2f} hours due to lack of available beds (warm-up period)")
        else:
            if self.experiment.trace:
                print(f"Elective Patient {patient_id} requests a critical care bed at {self.env.now:.2f} hours")
            with self.critical_care_beds.request() as req:
                yield req
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} admitted to a critical care bed at {self.env.now:.2f} hours")
                length_of_stay = self.experiment.rng_elective_treatment.exponential(self.experiment.elective_surgery_treatment_rate)
                yield self.env.timeout(length_of_stay)
                if self.env.now >= self.warmup_end_time:
                    self.total_treatment_time += length_of_stay
                if self.experiment.trace:
                    print(f"Elective Patient {patient_id} discharged from a critical care bed at {self.env.now:.2f} hours")
                yield self.env.timeout(self.experiment.intensive_cleaning_duration)
                if self.experiment.trace:
                    print(f"Intensive cleaning completed for Elective Patient {patient_id} at {self.env.now:.2f} hours")

    def run(self):
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())
        elective_surgery_process = self.env.process(self.elective_surgery_arrivals())
        warmup_complete_process = self.env.process(self.warmup_complete())
        self.env.run(until=self.experiment.total_run_length)

        # Calculate bed utilization as a proportion
        bed_utilization = self.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)

        # Calculate bed occupancy
        bed_occupancy = bed_utilization * self.experiment.num_critical_care_beds

        performance_measures = {
            "Total Cancelled Elective Operations": self.cancelled_operations,
            "Mean Unplanned Admission Waiting Time (hours)": self.total_unplanned_waiting_time / self.total_unplanned_admissions if self.total_unplanned_admissions > 0 else 0,
            "Bed Utilization": bed_utilization,
            "Bed Occupancy": bed_occupancy,
            "Patient Count": self.patient_id_counter
        }

        results_df = pd.DataFrame.from_dict(performance_measures, orient='index').T
        return results_df


In [14]:

# Example usage (modified by tester)
experiment = Experiment(trace=False)
results = multiple_replications(experiment, num_replications=5)
results_summary(results)

,mean,std
Total Cancelled Elective Operations,337.800000,38.745322
Mean Unplanned Admission Waiting Time (hours),62.453489,55.234108
Bed Utilization,0.909409,0.020721
Bed Occupancy,21.825821,0.497310
Patient Count,1650.400000,17.826946


## Testing

Here we do a number of regression tests to check that previous functionality is not affected by the `multiple_replications` function. 

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [17]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [18]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCU(env, experiment)
    results = ccu_model.run()
    return ccu_model, results

In [19]:
def extreme_value_test_1(extreme_value=M*5):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    experiment = Experiment(wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, elective_surgery_arrival_rate = extreme_value,
                            trace=False)
    return run_test(experiment)

In [20]:
model, results = extreme_value_test_1()
results

,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count
0,0.0,0.0,0.211865,5.084763,370.0


### Test 2: Extreme value test 2

PASS

In [21]:
def extreme_value_test_2(extreme_value=M*5):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(accident_emergency_arrival_rate = extreme_value, wards_arrival_rate = extreme_value,
                            emergency_surgery_arrival_rate = extreme_value, other_hospitals_arrival_rate = extreme_value,
                            xray_department_arrival_rate = extreme_value, trace=True)
    return run_test(experiment)

In [22]:
try:
    model, results = extreme_value_test_2()
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

Elective Patient 0 arrived at 9.52 hours
Elective Patient 1 requests a critical care bed at 9.52 hours
Elective Patient 1 admitted to a critical care bed at 9.52 hours
Elective Patient 1 arrived at 29.36 hours
Elective Patient 2 requests a critical care bed at 29.36 hours
Elective Patient 2 admitted to a critical care bed at 29.36 hours
Elective Patient 2 arrived at 45.46 hours
Elective Patient 3 requests a critical care bed at 45.46 hours
Elective Patient 3 admitted to a critical care bed at 45.46 hours
Elective Patient 2 discharged from a critical care bed at 50.28 hours
Intensive cleaning completed for Elective Patient 2 at 55.28 hours
Elective Patient 3 arrived at 69.11 hours
Elective Patient 4 requests a critical care bed at 69.11 hours
Elective Patient 4 admitted to a critical care bed at 69.11 hours
Elective Patient 1 discharged from a critical care bed at 78.99 hours
Elective Patient 4 arrived at 83.59 hours
Elective Patient 5 requests a critical care bed at 83.59 hours
Electiv

### Test 3: Different run length

PASS

In [25]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model, results = run_test(experiment)
    print(f'{model.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')
    return model.env.now

In [26]:
try:
    test_results_collection_period(10)
except ZeroDivisionError:
    print("caught zero division error due to extreme value test")

model.total_treatment_time=147.59775850998855
experiment.results_collection_period=10


### Test 4: Extreme value test 3

PASS.

In [29]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    model, results = run_test(experiment)
    print(f'{model.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')
    return model, results

In [30]:
model, results = extreme_value_test_3()
results

model.total_treatment_time=7457.194345913006
experiment.results_collection_period=8640


,Total Cancelled Elective Operations,Mean Unplanned Admission Waiting Time (hours),Bed Utilization,Bed Occupancy,Patient Count
0,479.0,4460.299269,0.863101,0.863101,1665.0


### Test 5: Extreme value test 4

PASS

In [31]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.

    Note: as patients do not leave beds, wait time, utilisation, and occupancy are not calculated.
    '''
    experiment = Experiment(accident_emergency_lognormal_mu=M, 
                 wards_lognormal_mu=M, 
                 emergency_surgery_lognormal_mu=M, 
                 other_hospitals_lognormal_mu=M, 
                 xray_department_lognormal_mu=M, 
                 elective_surgery_treatment_rate=M,
                 trace=True)
    return run_test(experiment)

In [32]:
try:
    extreme_value_test_4()
except ZeroDivisionError:
    print("caught expected zero division error due to extreme value test")

Patient 0 arrived from Accident and Emergency at 8.53 hours
Patient 1 from Accident and Emergency requests a critical care bed at 8.53 hours
Patient 1 from Accident and Emergency waited 0.00 hours
Patient 1 from Accident and Emergency admitted to a critical care bed at 8.53 hours
Elective Patient 1 arrived at 9.52 hours
Elective Patient 2 requests a critical care bed at 9.52 hours
Elective Patient 2 admitted to a critical care bed at 9.52 hours
Patient 2 arrived from Accident and Emergency at 21.60 hours
Patient 3 from Accident and Emergency requests a critical care bed at 21.60 hours
Patient 3 from Accident and Emergency waited 0.00 hours
Patient 3 from Accident and Emergency admitted to a critical care bed at 21.60 hours
Patient 3 arrived from Emergency Surgery at 29.00 hours
Patient 4 from Emergency Surgery requests a critical care bed at 29.00 hours
Patient 4 from Emergency Surgery waited 0.00 hours
Patient 4 from Emergency Surgery admitted to a critical care bed at 29.00 hours
Ele

### Test 6: Vary Warm-up parameter

PASS

In [33]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model, results = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [34]:
test_warmup_period(0)

(8640, 8640)